# <a id='preprocessing-data'></a> 6.3. [**Prétraitement des données**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_3_preprocessing.ipynb#preprocessing-data)</br>([*Preprocessing data*](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-data))

Le package `sklearn.preprocessing` fournit plusieurs fonctions utilitaires courantes et des classes de transformeurs pour changer les vecteurs de caractéristiques brutes en une représentation plus adaptée aux estimateurs aval.

En général, les algorithmes d'apprentissage bénéficient d'une standardisation de l'ensemble de données. Si des outliers sont présents dans l'ensemble, des redimensionneurs ou transformateurs robustes sont plus appropriés. Le comportement des différents redimensionneurs, transformeurs et normaliseurs sur un jeu de données contenant des outliers marginaux est mis en évidence dans [**Comparaison de l'effet de différents redimensionneurs sur des données avec des outliers**](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html).

# <a id='standardization-or-mean-removal-and-variance-scaling'></a> 6.3.1. Standardisation, ou suppression de la moyenne et normalisation de la variance

La **standardisation** des ensembles de données est **une exigence courante pour de nombreux estimateurs d'apprentissage automatique** mis en œuvre dans scikit-learn; ils pourraient mal se comporter si les caractéristiques individuelles ne ressemblent pas plus ou moins à des données distribuées suivant la distribution normale standard : Gaussienne avec une **moyenne nulle et une variance unitaire**.

En pratique, nous ignorons souvent la forme de la distribution et transformons simplement les données pour les centrer en supprimant la valeur moyenne de chaque caractéristique, puis en les dimensionnant en divisant les caractéristiques non constantes par leur écart-type.

Par exemple, de nombreux éléments utilisés dans la fonction objectif d'un algorithme d'apprentissage (tels que le noyau RBF des Machines à Vecteurs de Support ou les régulariseurs $\ell_1$ et $\ell_2$ des modèles linéaires) peuvent supposer que toutes les caractéristiques sont centrées autour de zéro ou ont une variance de même ordre. Si une caractéristique a une variance qui est des ordres de grandeur plus grande que les autres, elle pourrait dominer la fonction objectif et empêcher l'estimateur d'apprendre correctement des autres caractéristiques comme prévu.

Le module [**`preprocessing`**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) fournit la classe utilitaire [**`StandardScaler`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), qui est un moyen rapide et facile pour effectuer les opérations suivantes sur un jeu de données de type tableau :

In [1]:
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
scaler = preprocessing.StandardScaler().fit(X_train)
scaler
# StandardScaler()

StandardScaler()

In [2]:
scaler.mean_
# array([1. ..., 0. ..., 0.33...])

array([1.        , 0.        , 0.33333333])

In [3]:
scaler.scale_
#array([0.81..., 0.81..., 1.24...])

array([0.81649658, 0.81649658, 1.24721913])

In [4]:
X_scaled = scaler.transform(X_train)
X_scaled
# array([[ 0.  ..., -1.22...,  1.33...],
#        [ 1.22...,  0.  ..., -0.26...],
#        [-1.22...,  1.22..., -1.06...]])

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

Les données mises à l'échelle ont une moyenne nulle et une variance unitaire :

In [5]:
X_scaled.mean(axis=0)
# array([0., 0., 0.])

array([0., 0., 0.])

In [6]:
X_scaled.std(axis=0)
# array([1., 1., 1.])

array([1., 1., 1.])

Cette classe implémente l'API `Transformer` pour calculer la moyenne et l'écart-type sur un ensemble d'entraînement, afin de pouvoir réappliquer la même transformation sur l'ensemble de test. Cette classe est donc adaptée aux premières étapes d'un [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

In [7]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X, y = make_classification(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)  # apply scaling on training data
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('logisticregression', LogisticRegression())])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [8]:
pipe.score(X_test, y_test)  # apply scaling on testing data, without leaking training data.
# 0.96

0.96

Il est possible de désactiver le centrage ou la mise à l'échelle en passant respectivement les paramètres `with_mean=False` ou `with_std=False` au constructeur de [**`StandardScaler`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

## <a id='scaling-features-to-a-range'></a> 6.3.1.1. Mise à l'échelle des caractéristiques dans une plage donnée

Une autre méthode de standardisation consiste à mettre à l'échelle les caractéristiques pour ramener entre des valeurs minimale et maximale données, souvent entre zéro et un, ou de manière à ce que la valeur absolue maximale de chaque caractéristique soit mise à l'échelle à une taille unitaire. Cela peut être réalisé en utilisant [**`MinMaxScaler`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) ou [**`MaxAbsScaler`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html), respectivement.

La motivation pour utiliser cette mise à l'échelle comprend la robustesse par rapport aux écarts types très faibles des caractéristiques et la préservation des entrées nulles dans les données éparses.

Voici un exemple pour mettre à l'échelle une matrice de données factices dans la plage `[0, 1]` :

In [9]:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax
# array([[0.5       , 0.        , 1.        ],
#        [1.        , 0.5       , 0.33333333],
#        [0.        , 1.        , 0.        ]])

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

La même instance du transformeur peut ensuite être appliquée à de nouvelles données de test qui n'ont pas été vues lors de l'appel à la méthode `fit` : les mêmes opérations de mise à l'échelle et de translation seront appliquées pour assurer la cohérence avec la transformation effectuée sur les données d'entraînement.

In [10]:
X_test = np.array([[-3., -1.,  4.]])
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax
# array([[-1.5       ,  0.        ,  1.66666667]])

array([[-1.5       ,  0.        ,  1.66666667]])

Il est possible d'introspecter les attributs du redimensionneur pour en savoir plus sur la nature exacte de la transformation apprise sur les données d'entraînement.

In [11]:
min_max_scaler.scale_
# array([0.5       , 0.5       , 0.33...])

array([0.5       , 0.5       , 0.33333333])

In [12]:
min_max_scaler.min_
# array([0.        , 0.5       , 0.33...])

array([0.        , 0.5       , 0.33333333])

Si [**`MinMaxScaler`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) reçoit une plage explicite `feature_range=(min, max)`, la formule complète est :

In [ ]:
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min

[**`MaxAbsScaler`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html) fonctionne de manière très similaire, mais met à l'échelle de manière à ce que les données d'entraînement se situent dans la plage `[-1, 1]` en divisant par la plus grande valeur maximale de chaque caractéristique. Il est conçu pour les données qui sont déjà centrées sur zéro ou des données creuses.

Voici comment utiliser les données factices de l'exemple précédent avec ce redimensionneur :

In [13]:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
max_abs_scaler = preprocessing.MaxAbsScaler()
X_train_maxabs = max_abs_scaler.fit_transform(X_train)
X_train_maxabs
# array([[ 0.5, -1. ,  1. ],
#        [ 1. ,  0. ,  0. ],
#        [ 0. ,  1. , -0.5]])

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

In [14]:
X_test = np.array([[ -3., -1.,  4.]])
X_test_maxabs = max_abs_scaler.transform(X_test)
X_test_maxabs
# array([[-1.5, -1. ,  2. ]])

array([[-1.5, -1. ,  2. ]])

In [15]:
max_abs_scaler.scale_
# array([2.,  1.,  2.])

array([2., 1., 2.])

# <a id='encoding-categorical-features'></a> 6.3.4. Encodage des caractéristiques catégorielles

Souvent, les caractéristiques ne sont pas données sous forme de valeurs continues mais catégorielles. Par exemple, une personne peut avoir des caractéristiques `["homme", "femme"]`, `["from Europe", "from États-Unis", "from Asia"]`, `["uses Firefox", "uses Chrome", "uses Safari", "uses Internet Explorer"]`. De telles caractéristiques peuvent être efficacement codées sous forme d'entiers, par exemple `["male", "from États-Unis", "uses Internet Explorer"]` pourrait être exprimé par `[0, 1, 3]` tandis que `["female", "from Asia", "uses Chrome"]` serait `[1, 2, 1]`.

Pour convertir des caractéristiques catégorielles en de tels codes entiers, nous pouvons utiliser [**`OrdinalEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html). Cet estimateur transforme chaque caractéristique catégorielle en une nouvelle caractéristique d'entiers ($0$ à $n_{\text{categories}} - 1$) :

In [ ]:
from sklearn import preprocessing
enc = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
# OrdinalEncoder()
enc.transform([['female', 'from US', 'uses Safari']])
# array([[0., 1., 1.]])

array([[0., 1., 1.]])

Une telle représentation entière ne peut cependant pas être utilisée directement avec tous les estimateurs scikit-learn, car ceux-ci attendent une entrée continue et interpréteraient les catégories en tant qu'ordinaux, ce qui n'est souvent pas souhaité (c'est-à-dire que l'ensemble des navigateurs a été ordonné arbitrairement).

Par défaut, [**`OrdinalEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) transmettra également les valeurs manquantes représentées par `np.nan`.

In [ ]:
import numpy as np
enc = preprocessing.OrdinalEncoder()
X = [['male'], ['female'], [np.nan], ['female']]
enc.fit_transform(X)
# array([[ 1.],
#       [ 0.],
#       [nan],
#       [ 0.]])

array([[ 1.],
       [ 0.],
       [nan],
       [ 0.]])

[**`OrdinalEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) fournit un paramètre `encoded_missing_value` pour encoder les valeurs manquantes sans avoir besoin de créer un pipeline et en utilisant [**`SimpleImputer`**](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer).

In [ ]:
enc = preprocessing.OrdinalEncoder(encoded_missing_value=-1)
X = [['male'], ['female'], [np.nan], ['female']]
enc.fit_transform(X)
# array([[ 1.],
#        [ 0.],
#        [-1.],
#        [ 0.]])

array([[ 1.],
       [ 0.],
       [-1.],
       [ 0.]])

Le traitement ci-dessus est équivalent au pipeline suivant :

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
enc = Pipeline(steps=[
        ("encoder", preprocessing.OrdinalEncoder()),
        ("imputer", SimpleImputer(strategy="constant", fill_value=-1)),
    ])
enc.fit_transform(X)
# array([[ 1.],
#        [ 0.],
#        [-1.],
#        [ 0.]])

Une autre possibilité de convertir des caractéristiques catégorielles en caractéristiques pouvant être utilisées avec les estimateurs scikit-learn consiste à utiliser un one-of-K, également connu sous le nom de codage one-hot ou factice. Ce type d'encodage peut être obtenu avec le [**`OneHotEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), qui transforme chaque caractéristique catégorique avec `n_categories` valeurs possibles en `n_categories` caractéristiques binaires, avec l'une d'elles à 1, et toutes les autres à 0.

Reprenons l'exemple ci-dessus :

In [ ]:
enc = preprocessing.OneHotEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
# OneHotEncoder()
enc.transform([['female', 'from US', 'uses Safari'],
               ['male', 'from Europe', 'uses Safari']]).toarray()
# array([[1., 0., 0., 1., 0., 1.],
#        [0., 1., 1., 0., 0., 1.]])

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 1.]])

Par défaut, les valeurs que chaque caractéristique peut prendre sont automatiquement déduites du jeu de données et peuvent être trouvées dans l'attribut `categories_` :

In [ ]:
enc.categories_
# [array(['female', 'male'], dtype=object),
#  array(['from Europe', 'from US'], dtype=object),
#  array(['uses Firefox', 'uses Safari'], dtype=object)]

[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

Il est possible de le spécifier explicitement à l'aide du paramètre `categories`. Il existe deux genres, quatre continents possibles et quatre navigateurs Web dans notre ensemble de données :

In [ ]:
genders = ['female', 'male']
locations = ['from Africa', 'from Asia', 'from Europe', 'from US']
browsers = ['uses Chrome', 'uses Firefox', 'uses IE', 'uses Safari']
enc = preprocessing.OneHotEncoder(categories=[genders, locations, browsers])
# Note that for there are missing categorical values for the 2nd and 3rd
# feature
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
# OneHotEncoder(categories=[['female', 'male'],
#                           ['from Africa', 'from Asia', 'from Europe',
#                            'from US'],
#                           ['uses Chrome', 'uses Firefox', 'uses IE',
#                            'uses Safari']])
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()
# array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])

array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])

S'il est possible que les données d'entraînement aient des caractéristiques catégorielles manquantes, il peut souvent être préférable de spécifier `handle_unknown='infrequent_if_exist'` au lieu de définir les catégories manuellement comme ci-dessus. Lorsque `handle_unknown='infrequent_if_exist'` est spécifié et que des catégories inconnues sont rencontrées lors de la transformation, aucune erreur n'est générée, mais les colonnes encodées à chaud résultantes pour cette caractéristique sont toutes nulles ou considérées comme une catégorie peu fréquente si elle cette option est activée. (`handle_unknown='infrequent_if_exist'` n'est pris en charge que pour l'encodage one-hot) :

In [ ]:
enc = preprocessing.OneHotEncoder(handle_unknown='infrequent_if_exist')
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
# OneHotEncoder(handle_unknown='infrequent_if_exist')
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()
# array([[1., 0., 0., 0., 0., 0.]])

array([[1., 0., 0., 0., 0., 0.]])

Il est également possible d'encoder chaque colonne en `n_categories - 1` colonnes au lieu de `n_categories` colonnes en utilisant le paramètre `drop`. Ce paramètre permet à l'utilisateur de spécifier une catégorie pour chaque caractéristique à supprimer. Ceci est utile pour éviter la colinéarité dans la matrice d'entrée dans certains classificateurs. Une telle fonctionnalité est utile, par exemple, lors de l'utilisation d'une régression non régularisée ([**`LinearRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)), car la colinéarité rendrait la matrice de covariance non inversible :

In [ ]:
X = [['male', 'from US', 'uses Safari'],
     ['female', 'from Europe', 'uses Firefox']]
drop_enc = preprocessing.OneHotEncoder(drop='first').fit(X)
drop_enc.categories_
# [array(['female', 'male'], dtype=object), array(['from Europe', 'from US'], dtype=object),
#  array(['uses Firefox', 'uses Safari'], dtype=object)]
drop_enc.transform(X).toarray()
# array([[1., 1., 1.],
#        [0., 0., 0.]])

array([[1., 1., 1.],
       [0., 0., 0.]])

On peut vouloir supprimer l'une des deux colonnes uniquement pour les caractéristiques à 2 catégories. Dans ce cas, vous pouvez définir le paramètre `drop='if_binary'`.

In [ ]:
X = [['male', 'US', 'Safari'],
     ['female', 'Europe', 'Firefox'],
     ['female', 'Asia', 'Chrome']]
drop_enc = preprocessing.OneHotEncoder(drop='if_binary').fit(X)
drop_enc.categories_
# [array(['female', 'male'], dtype=object), array(['Asia', 'Europe', 'US'], dtype=object),
#  array(['Chrome', 'Firefox', 'Safari'], dtype=object)]
drop_enc.transform(X).toarray()
# array([[1., 0., 0., 1., 0., 0., 1.],
#        [0., 0., 1., 0., 0., 1., 0.],
#        [0., 1., 0., 0., 1., 0., 0.]])

array([[1., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0.]])

Dans le `X` transformé, la première colonne est encode la caractéristique avec les catégories "masculin"/"féminin", tandis que les 6 colonnes restantes encodent les 2 caractéristiques avec respectivement 3 catégories chacune.

Lorsque `handle_unknown='ignore'` et que `drop` n'est pas `None`, les catégories inconnues seront encodées en zéros :

In [ ]:
drop_enc = preprocessing.OneHotEncoder(drop='first',
                                       handle_unknown='ignore').fit(X)
X_test = [['unknown', 'America', 'IE']]
drop_enc.transform(X_test).toarray()
# array([[0., 0., 0., 0., 0.]])

Toutes les catégories dans `X_test` sont inconnues lors de la transformation et seront mappées sur tous les zéros. Cela signifie que les catégories inconnues auront la même correspondance que la catégorie supprimée. : `OneHotEncoder.inverse_transform` mappera tous les zéros sur la catégorie supprimée si une catégorie est supprimée et `None` si une catégorie n'est pas supprimée :

In [ ]:
drop_enc = preprocessing.OneHotEncoder(drop='if_binary', sparse=False,
                                       handle_unknown='ignore').fit(X)
X_test = [['unknown', 'America', 'IE']]
X_trans = drop_enc.transform(X_test)
X_trans
# array([[0., 0., 0., 0., 0., 0., 0.]])
drop_enc.inverse_transform(X_trans)
# array([['female', None, None]], dtype=object)

c:\Users\franc\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array([['female', None, None]], dtype=object)

[**`OneHotEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) prend en charge les caractéristiques catégorielles avec des valeurs manquantes en considérant les valeurs manquantes comme une catégorie supplémentaire :

In [ ]:
X = [['male', 'Safari'],
     ['female', None],
     [np.nan, 'Firefox']]
enc = preprocessing.OneHotEncoder(handle_unknown='error').fit(X)
enc.categories_
# [array(['female', 'male', nan], dtype=object),
#  array(['Firefox', 'Safari', None], dtype=object)]
enc.transform(X).toarray()
# array([[0., 1., 0., 0., 1., 0.],
#        [1., 0., 0., 0., 0., 1.],
#        [0., 0., 1., 1., 0., 0.]])

array([[0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.]])

Si une caractéristique contient à la fois `np.nan` et `None`, elles seront considérées comme des catégories distinctes :

In [ ]:
X = [['Safari'], [None], [np.nan], ['Firefox']]
enc = preprocessing.OneHotEncoder(handle_unknown='error').fit(X)
enc.categories_
# [array(['Firefox', 'Safari', None, nan], dtype=object)]
enc.transform(X).toarray()
# array([[0., 1., 0., 0.],
#        [0., 0., 1., 0.],
#        [0., 0., 0., 1.],
#        [1., 0., 0., 0.]])

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

Voir [**Chargement de caractéristiques** (6.2.1)](https://scikit-learn.org/stable/modules/feature_extraction.html#dict-feature-extraction) à partir de dicts pour des caractéristiques catégorielles représentées sous forme de dictionnaires, et non sous forme de scalaires.

## <a id='infrequent-categories'></a> 6.3.4.1. Catégories peu fréquentes

[**`OneHotEncoder`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) prend en charge l'agrégation de catégories peu fréquentes en une seule sortie pour chaque caractéristique. Les paramètres permettant de rassembler les catégories peu fréquentes sont `min_frequency` et `max_categories`.

1. `min_frequency` est soit un entier supérieur ou égal à 1, soit un flottant dans l'intervalle `(0.0, 1.0)`. Si `min_frequency` est un entier, les catégories avec une cardinalité inférieure à `min_frequency` seront considérées comme peu fréquentes. Si `min_frequency` est un flottant, les catégories avec une cardinalité inférieure à cette fraction du nombre total d'échantillons seront considérées comme peu fréquentes. La valeur par défaut est 1, ce qui signifie que chaque catégorie est encodée séparément.

2. `max_categories` vaut `None` ou tout entier supérieur à 1. Ce paramètre définit une limite supérieure au nombre de caractéristiques en sortie pour chaque caractéristique en entrée. `max_categories` inclut la caractéristique qui combine des catégories peu fréquentes.

Dans l'exemple suivant, les catégories `dog`, `snake` sont considérées comme peu fréquentes :

In [ ]:
X = np.array([['dog'] * 5 + ['cat'] * 20 + ['rabbit'] * 10 +
              ['snake'] * 3], dtype=object).T
enc = preprocessing.OneHotEncoder(min_frequency=6, sparse=False).fit(X)
enc.infrequent_categories_
# [array(['dog', 'snake'], dtype=object)]
enc.transform(np.array([['dog'], ['cat'], ['rabbit'], ['snake']]))
# array([[0., 0., 1.],
#        [1., 0., 0.],
#        [0., 1., 0.],
#        [0., 0., 1.]])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

En définissant `handle_unknown` sur `'infrequent_if_exist'`, les catégories inconnues seront considérées comme peu fréquentes :

In [ ]:
enc = preprocessing.OneHotEncoder(
    handle_unknown='infrequent_if_exist', sparse=False, min_frequency=6)
enc = enc.fit(X)
enc.transform(np.array([['dragon']]))
# array([[0., 0., 1.]])

array([[0., 0., 1.]])

[**`OneHotEncoder.get_feature_names_out`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder.get_feature_names_out) utilise `'infrequent'` pour désigner les caractéristiques peu fréquentes :

In [ ]:
enc.get_feature_names_out()
# array(['x0_cat', 'x0_rabbit', 'x0_infrequent_sklearn'], dtype=object)

array(['x0_cat', 'x0_rabbit', 'x0_infrequent_sklearn'], dtype=object)

Lorsque `'handle_unknown'` est défini sur `'infrequent_if_exist'` et qu'une catégorie inconnue est rencontrée dans la transformation :

1. Si la prise en charge de la catégorie peu fréquente n'a pas été configurée ou s'il n'y avait pas de catégorie peu fréquente pendant l'entraînement, les colonnes encodées à chaud résultantes pour cette caractéristique seront toutes des zéros. Dans la transformation inverse, une catégorie inconnue sera notée `None`.

2. S'il y a une catégorie peu fréquente pendant l'entraînement, la catégorie inconnue sera considérée comme peu fréquente. Dans la transformation inverse, "infrequent_sklearn" sera utilisé pour représenter la catégorie peu fréquente.

Les catégories peu fréquentes peuvent également être configurées à l'aide de `max_categories`. Dans l'exemple suivant, nous définissons `max_categories=2` pour limiter le nombre de caractéristiques dans la sortie. Il en résultera que toutes les catégories sauf la catégorie `'cat'` seront considérées comme peu fréquentes, ce qui conduira à deux caractéristiques, une pour les catégories `'cat'` et une pour les catégories peu fréquentes - qui sont toutes les autres :

In [ ]:
enc = preprocessing.OneHotEncoder(max_categories=2, sparse=False)
enc = enc.fit(X)
enc.transform([['dog'], ['cat'], ['rabbit'], ['snake']])
# array([[0., 1.],
#        [1., 0.],
#        [0., 1.],
#        [0., 1.]])

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]])

Si `max_categories` et `min_frequency` ne sont pas des valeurs par défaut, les catégories sont sélectionnées en fonction de `min_frequency` en premier et les catégories `max_categories` sont conservées. Dans l'exemple suivant, `min_frequency=4` considère que seul `snake` est peu fréquent, mais `max_categories=3`, force `dog` à être également peu fréquent :

In [ ]:
enc = preprocessing.OneHotEncoder(min_frequency=4, max_categories=3, sparse=False)
enc = enc.fit(X)
enc.transform([['dog'], ['cat'], ['rabbit'], ['snake']])
# array([[0., 0., 1.],
#        [1., 0., 0.],
#        [0., 1., 0.],
#        [0., 0., 1.]])

S'il existe des catégories peu fréquentes avec la même cardinalité au seuil de `max_categories`, alors les premières `max_categories` sont prises en fonction de l'ordre du lexique. Dans l'exemple suivant, « b », « c » et « d » ont la même cardinalité et avec `max_categories=2`, « b » et « c » sont peu fréquents car ils ont un ordre de lexique plus élevé.

In [ ]:
X = np.asarray([["a"] * 20 + ["b"] * 10 + ["c"] * 10 + ["d"] * 10], dtype=object).T
enc = preprocessing.OneHotEncoder(max_categories=3).fit(X)
enc.infrequent_categories_
# [array(['b', 'c'], dtype=object)]

[array(['b', 'c'], dtype=object)]

## 6.3.5. Discrétisation

...